# Nivel 1
Conexión a base de datos y ejecución de querys.

In [1]:
# import library to connect with db
import psycopg2

In [2]:
# Connect with postgres db
try:
    conn = psycopg2.connect(port=5432, host="localhost", database="MeLi",
                            user="MeLi", password="MeLi")
except Exception as e:
    print(e)

## Query 1

¿Cuál es la hora del día en que se realizan más búsquedas en MercadoLibre?

In [3]:
# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("""
SELECT extract(hour from timestamp) as hour
FROM dataset_meli
WHERE event_name = 'SEARCH'
GROUP BY hour
ORDER BY count(*) desc
LIMIT 1;
""")

# Retrieve query results
query_1 = cur.fetchall()
print(f'Hora con más frecuencia de búsquedas: {query_1[0][0]} hs.')

Hora con más frecuencia de búsquedas: 23 hs.


## Query 2

¿Cuál fue el experimento que tuvo más participantes dentro del dataset?

### Opción 1

In [4]:
# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("""
SELECT SPLIT_PART(experiments_, '=', 1) as splitted_exp, count(*) as count
FROM (  SELECT unnest(string_to_array(experiments_, ',')) as experiments_
        FROM (  SELECT REPLACE((
                SELECT REPLACE((
                SELECT REPLACE(experiments,', ',',')
                ),'}','')
                ),'{','') as experiments_ FROM dataset_meli) table_) table_
GROUP BY splitted_exp
ORDER BY count DESC
LIMIT 1;
""")

# Retrieve query results
query_2 = cur.fetchall()
print(f'Experimentos con más resultados: {query_2[0][0]}. \nCon un total de {query_2[0][1]} resultados.')

Experimentos con más resultados: cookiesConsentBanner. 
Con un total de 135269 resultados.


### Opción 2

In [5]:
# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("""
ALTER TABLE dataset_meli
ADD COLUMN experiments_json json;

UPDATE 
   dataset_meli
SET 
   experiments_json =  CAST( REPLACE((
       SELECT REPLACE((
       SELECT REPLACE((
       SELECT REPLACE(experiments, ', ','", "')), '{','{"')), '}','"}')), '=','" : "') AS JSON);

WITH experiments AS (
        SELECT json_object_keys(experiments_json) experiment 
        FROM dataset_meli)
SELECT experiment, COUNT(*) as count 
FROM experiments 
GROUP BY experiment
ORDER BY count desc
LIMIT 1;
""")

# Retrieve query results
query_3 = cur.fetchall()
print(f'Experimentos con más resultados:{query_3[0][0]}. \nCon un total de {query_3[0][1]} resultados.')

Experimentos con más resultados:cookiesConsentBanner. 
Con un total de 135269 resultados.


# Nivel 3

Hostear el dataset en algún motor de consulta SQL cloud de preferencia ( MySQL,
RedShift, BigQuery, Athena, SparkSQL, Azure SQL DataWarehouse, etc ) y disponibilizar
instrucciones para su conexión remota y permitir ejecutar consultas SQL.

In [6]:
# Import google libraries
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd

# Path to your json key file
KEY_PATH = "meli-351904-bbaa7ed94ff7.json"

# Read the credentials from our file
# Scopes are not necessary because we defined them in GCP already
CREDS = service_account.Credentials.from_service_account_file(KEY_PATH)

# The client object will be used to interact with BQ
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)

In [7]:
# SQL Query
QUERY = """
SELECT * 
FROM `meli-351904.experiments_ds.dataset`
"""

# Execute query and transform to dataframe 
result_query = (
    client.query(QUERY)
    .result()
    .to_dataframe()
)

# Print information
result_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141557 entries, 0 to 141556
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   event_name   141554 non-null  object             
 1   item_id      64803 non-null   Int64              
 2   timestamp    141554 non-null  datetime64[ns, UTC]
 3   site         141554 non-null  object             
 4   experiments  141554 non-null  object             
 5   user_id      141554 non-null  Int64              
dtypes: Int64(2), datetime64[ns, UTC](1), object(3)
memory usage: 6.8+ MB


## Explore original dataset with pandas

In [8]:
# Load original dataset
df = pd.read_csv("dataset.csv")

# Print info, to verify results
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141554 entries, 0 to 141553
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   event_name   141554 non-null  object 
 1   item_id      64803 non-null   float64
 2   timestamp    141554 non-null  object 
 3   site         141554 non-null  object 
 4   experiments  141554 non-null  object 
 5   user_id      141554 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 6.5+ MB
